# DM - Système de décision

## PARTIE 1 : Gurobi

In [23]:
import os
import random
from time import perf_counter

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from gurobipy import Model
from gurobipy.gurobipy import GRB
from numpy.random import normal
from sklearn.metrics import confusion_matrix

from generate_dataset import convert_dataset, generate_dataset

pio.templates.default = "plotly_white"

On génère un dataset qui pourra être résolu par notre solveur. 

Il nous faut donc des valeurs cohérentes de notes par rapport aux coefficients des différentes matières ainsi que des frontières entre les différentes classes (Accepté, Refusé, ...).

In [24]:
X = convert_dataset(*generate_dataset(10, 4, 3), 3)


Les coeffs sont =  [0.27118644 0.38983051 0.18644068 0.15254237]

Les frontières sont =  [[12. 17.]
 [ 2. 13.]
 [ 6.  8.]
 [ 9. 15.]]

lambda =  0.820930125907707

Les notes des élèves :
[[13  8  2  8]
 [ 7  0  7 11]
 [12  2 10 13]
 [18 20 14  3]
 [17  2 14  0]
 [12 20  4  0]
 [18 19  7 20]
 [15 11 10  5]
 [11 13 14  0]
 [ 0 15  6  8]]

Les classes des élèves :
[0, 0, 1, 2, 1, 0, 1, 1, 0, 0]


In [25]:
def solve(X, students_count, subjects_count, classes_count, verbose=0):
    """
    :param X: {list(np.array()[]} Liste de classes_count matrices de notes
    :param students_count: {int} Nombre d'étudiants
    :param subjects_count: {int} Nombre de matières
    :param classes_count:  {int} Nombre de classes (ex: 2 pour {Accepté, Refusé})
    :param verbose: {int} 0: prints nothing, 1:prints results, 2: debug level
    :return:
    """
    R, *L, A = X

    # Instanciation du modèle
    m = Model("PL modeling")

    #------------------VARIABLES------------------

    # Vecteur des coefficients des différentes matières
    w = m.addMVar(shape=subjects_count, lb=0, ub=1, name="w")

    c_accepted = m.addMVar(shape=(len(A), subjects_count), lb=0, ub=1, name="c_accepted")
    c_refused = m.addMVar(shape=(len(R), subjects_count), lb=0, ub=1, name="c_refused")

    delta_accepted = m.addMVar(shape=(students_count, subjects_count), vtype=GRB.INTEGER, lb=0, ub=1,
                               name="delta_accepted")
    delta_refused = m.addMVar(shape=(students_count, subjects_count), vtype=GRB.INTEGER, lb=0, ub=1,
                              name="delta_refused")

    sigma_accepted = m.addMVar(shape=len(A), name="x", vtype=GRB.CONTINUOUS)
    sigma_refused = m.addMVar(shape=len(R), name="y", vtype=GRB.CONTINUOUS)

    lambd = m.addVar(name="lambda", lb=0.5, ub=1)

    b = m.addMVar(shape=subjects_count, name="b", lb=0, ub=20)

    alpha = m.addVar(name="alpha")

    m.update()

    epsilon = 0.01

    M = 150

    #------------------CONTRAINTES------------------

    # Validation constraint with sigma
    for index, student_point in enumerate(c_accepted):
        m.addConstr(sum(student_point) - sigma_accepted[index] + epsilon == lambd)

    # Refusal constraint with sigma
    for index, student_point in enumerate(c_refused):
        m.addConstr(sum(student_point) == lambd - sigma_refused[index])

    # Alpha is inferior than all sigmas
    for offset in (*sigma_accepted, *sigma_refused):
        m.addConstr(alpha <= offset)

    accepted_tuple = (c_accepted, delta_accepted, A)
    refused_tuple = (c_refused, delta_refused, R)

    for c, delta, grades in (accepted_tuple, refused_tuple):
        for i in range(c.shape[0]):
            for j in range(c.shape[1]):
                m.addConstr(c[i, j] <= w[j])
                m.addConstr(c[i, j] <= delta[i, j])
                m.addConstr(c[i, j] >= delta[i, j] - 1 + w[j])
                m.addConstr(M * delta[i, j] + epsilon >= grades[i, j] - b[j])
                m.addConstr(M * (delta[i, j] - 1) <= grades[i, j] - b[j])

    m.addConstr(sum(w) == 1)

    m.update()

    if verbose == 0:
        m.params.outputflag = 0

    #Fonction objectif
    m.setObjective(alpha, GRB.MAXIMIZE)
    m.update()

    m.optimize()
    if verbose >= 1:
        print("====== Model solved =====")
        print(f"alpha = {alpha.X}")
        print(f"lambda = {lambd.X}")
        print(f"b = {b.X}")

        print("\nLes coefficients de notre solution optimale sont :")
        print(f"__________________")

        print(f"| Matière  |coeff|")

        for index, coef in enumerate(w.X):
            print(f"|matière {index + 1} | {coef.round(2)} |")
        print(f"__________________")

    return A, R, w.X, lambd.X, b.X


### Exemple 1 : 

20 élèves, 5 matières et 2 classes (Accepté ou Refusé) 

In [26]:
X = convert_dataset(*generate_dataset(20, 5, 2), 2)


Les coeffs sont =  [0.07017544 0.33333333 0.31578947 0.0877193  0.19298246]

Les frontières sont =  [[ 3.]
 [ 6.]
 [13.]
 [17.]
 [13.]]

lambda =  0.8116400429237574

Les notes des élèves :
[[19  6 10 17  3]
 [ 9 20  2 11  8]
 [ 1  1 14 11 12]
 [ 9  9  9 18  2]
 [ 5 12 17  8  7]
 [10  4  4  4 17]
 [19  0 20 13  4]
 [11 10 15 20 12]
 [20 10  8  7  1]
 [ 4  8 18 12  5]
 [16 17 18  5 10]
 [ 1  8  2  5 12]
 [13 13 20 12 16]
 [19  7 12  7 18]
 [ 8  5  2  2  5]
 [14  0 17  2  2]
 [ 9 15  1 19  8]
 [ 4 19  6 20 17]
 [14 19  7  0 17]
 [ 7 17 14 15  3]]

Les classes des élèves :
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [27]:
accepted, refused, weigths, lambd, b = solve(X, 20, 5, 2, verbose=2)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 541 rows, 332 columns and 1285 nonzeros
Model fingerprint: 0x94cc7c85
Variable types: 132 continuous, 200 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-01, 2e+01]
  RHS range        [1e-02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 34 rows and 120 columns
Presolve time: 0.00s
Presolved: 507 rows, 212 columns, 1217 nonzeros
Variable types: 107 continuous, 105 integer (100 binary)

Root relaxation: objective 5.050000e-01, 177 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.5050000    0.50500  0.00%     -    0s

Explored 1 nodes (379 simplex i

In [28]:
print((accepted >= b) @ weigths.T >= lambd)

[ True]


In [29]:
print((refused >= b) @ weigths.T >= lambd)

[False False False False False False  True False False False False False
 False False False False False False False]


**Conclusion :**

Avec 2 classes (Accepté, Refusé), notre MRSort fonctionne et nous renvoie une solution faisable pour notre dataset.

# Influence du nombre d'instance

In [30]:
def compute_performance(test_dataset, stats_df, parameter_name, parameter_value, noise=0):
    """
    Calcul les performance d'un modèle de poids sur un dataset de test donné
    :param test_dataset: {(np.array(), classe} Dataset de test
    :param stats_df: {pd.Dataframe} Dataframe dans lequel les stats seront ajoutées
    :param parameter_name: {str} Nom du paramètre évalué
    :param parameter_value: {int} Valeur du paramètre évalué
    :return: {pd.Dataframe} Dataframe contenant les statistiques du modèle
    """
    X_test, y_test = list(zip(*test_dataset))

    X_test_noised = np.array(X_test) + normal(0, noise, np.array(X_test).shape)

    pred = (X_test_noised >= b) @ weights.T >= lambd

    confusion_mat = confusion_matrix(y_test, pred, labels=[0, 1])
    TP = confusion_mat[0, 0]
    TN = confusion_mat[1, 1]
    FP = confusion_mat[0, 1]
    FN = confusion_mat[1, 0]

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    duration = end - start
    raw_stats = {
        "precision": precision,
        "recall": recall,
        "accuracy": (TP + TN) / (TP + TN + FP + FN),
        "F1 score": 2 * (precision * recall) / (precision + recall),
        "duration (in s)": duration
    }

    for key, value in raw_stats.items():
        stats_df = stats_df.append({"type": key, "value": value, parameter_name: parameter_value},
                                   ignore_index=True)

    return stats_df, duration


In [31]:
graphs_directories = "graphs"

elapsed_time = 0
students_count = 10
subjects_count = 4
classes_count = 2
stats = pd.DataFrame(columns=["type", "value", "students_count"])

while elapsed_time < 12:
    print(students_count, end=" ")
    for i in range(12):
        dataset = generate_dataset(students_count * 2, subjects_count, classes_count, verbose=0)
        dataset = list(zip(list(dataset[0]), dataset[1]))

        random.shuffle(dataset)
        train = dataset[:len(dataset) // 2]
        test = dataset[len(dataset) // 2:]

        train = convert_dataset(*list(zip(*train)), classes_count)

        start = perf_counter()
        accepted, refused, weights, lambd, b = solve(train, students_count, subjects_count,
                                                     classes_count, verbose=0)
        end = perf_counter()

        stats, elapsed_time = compute_performance(test, stats, "students_count", students_count)

    students_count += 50
aggregated_stats = stats.groupby(["students_count", "type"]).agg({"value": "mean"}).reset_index()

10 60 110 160 210 

KeyboardInterrupt: 

In [ ]:
fig = px.line(aggregated_stats[~(aggregated_stats["type"] == "duration (in s)")], x="students_count", y="value",
              color="type",
              title=f"Model performance depending on students_count with subjects_counts={subjects_count}")

if not os.path.exists(graphs_directories):
    os.mkdir(graphs_directories)
fig.show()

try:
    fig.write_image(f"{graphs_directories}/gurobi_perf_students.png")
except:
    print("Please install kaleido (pip install -U kaleido for static image export)")

In [ ]:
fig = px.line(aggregated_stats[(aggregated_stats["type"] == "duration (in s)")], x="students_count", y="value",
              color="type", title=f"Training duration depending on students_count with subject_counts={subjects_count}")
fig.show()
try:
    fig.write_image(f"{graphs_directories}/gurobi_duration_students.png")
except:
    print("Please install kaleido (pip install -U kaleido for static image export)")

# Influence du nombre de critères

In [ ]:
stats = pd.DataFrame(columns=["type", "value", "subjects_count"])

subjects_count = 2
students_count = 600
elapsed_time = 0

while elapsed_time < 20:
    for i in range(12):
        dataset = generate_dataset(students_count * 2, subjects_count, classes_count, verbose=0)
        dataset = list(zip(list(dataset[0]), dataset[1]))

        random.shuffle(dataset)
        train = dataset[:len(dataset) // 2]
        test = dataset[len(dataset) // 2:]

        train = convert_dataset(*list(zip(*train)), classes_count)

        start = perf_counter()
        accepted, refused, weights, lambd, b = solve(train, students_count, subjects_count,
                                                     classes_count, verbose=0)
        end = perf_counter()

        stats, elapsed_time = compute_performance(test, stats, "subjects_count", subjects_count)

    subjects_count += 1
aggregated_stats = stats.groupby(["subjects_count", "type"]).agg({"value": "mean"}).reset_index()

In [ ]:
fig = px.line(aggregated_stats[~(aggregated_stats["type"] == "duration (in s)")], x="subjects_count", y="value",
              color="type", title=f"Model performance depending on subjects_count with students_count={students_count}")

fig.show()
try:
    fig.write_image(f"{graphs_directories}/gurobi_perf_subjects.png")
except:
    print("Please install kaleido (pip install -U kaleido for static image export)")

In [ ]:
fig = px.line(aggregated_stats[(aggregated_stats["type"] == "duration (in s)")], x="subjects_count", y="value",
              color="type",
              title=f"Training duration depending on subject_counts with students_counts={students_count}")
fig.show()
try:
    fig.write_image(f"{graphs_directories}/gurobi_duration_subjects.png")
except:
    print("Please install kaleido (pip install -U kaleido for static image export)")

# Influence du bruit

In [ ]:
stats = pd.DataFrame(columns=["type", "value", "subjects_count"])

subjects_count = 4
students_count = 500
elapsed_time = 0

for noise in np.arange(0, 20, 0.3):
    for i in range(8):
        if i == 0:
            print(f"{noise:.1f}", end=" ")
        dataset = generate_dataset(students_count * 2, subjects_count, classes_count, verbose=0)
        dataset = list(zip(list(dataset[0]), dataset[1]))

        random.shuffle(dataset)
        train = dataset[:len(dataset) // 2]
        test = dataset[len(dataset) // 2:]

        train = convert_dataset(*list(zip(*train)), classes_count)

        start = perf_counter()
        accepted, refused, weights, lambd, b = solve(train, students_count, subjects_count, classes_count, verbose=0)
        end = perf_counter()
        stats, elapsed_time = compute_performance(dataset, stats, "écart-type", noise, noise=noise)
aggregated_stats = stats.groupby(["écart-type", "type"]).agg({"value": "mean"}).reset_index()

In [ ]:
fig = px.line(aggregated_stats[~(aggregated_stats["type"] == "duration (in s)")], x="écart-type", y="value",
              color="type",
              title=f"Model performance with noise applied thanks to a centered normal <br> law with students_count={students_count} and subjects_count={subjects_count}")

fig.show()
try:
    fig.write_image(f"{graphs_directories}/gurobi_perf_noise.png")
except:
    print("Please install kaleido (pip install -U kaleido) for static image export")